In [1]:
import numpy as np
from scipy.spatial.distance import pdist, cdist
import matplotlib.pyplot as plt

In [2]:
# Plotting.
def show(data):
    plt.imshow(data)
    plt.colorbar()
    plt.show()

In [3]:
# Kernel function stuff...
def rbf_kernel(A, B, scale=1.0):
    sq_dists = cdist(A, B, metric='euclidean')
    return np.exp(-sq_dists / (2 * scale ** 2))

In [4]:
# Testing.
# Get n x 2 coordiante matrix D.
def get_coordinates_uniform(radius, num_each_dir):
    x = np.linspace(-radius, radius, num_each_dir)
    y = x.copy()
    xg, yg = np.meshgrid(y, x)
    xf, yf = xg.flatten(), yg.flatten()
    D = np.column_stack([xf, yf])
    return D

# Get f(x, y), f_x + noise, f_y + noise.
def test_surface_data(D, noise):
    # Surface is z = x^2 + y^2.
    xf, yf = D[:, 0], D[:, 1]
    zf = 0.1 * (xf ** 2 + yf ** 2)

    # Get noisy gradient data.
    zxf = 0.2 * xf + noise * np.random.randn(len(xf))
    zyf = 0.2 * yf + noise * np.random.randn(len(yf))
    g = np.concatenate([zxf, zyf])

    # Return true surface and noisy gradient as vectors.
    return [zf, g]

# Get stuff needed for the regression.
# For right now, assume Kernel is RBF.

# Build matrix theta_ij = phi_i(phi_j(K((x1, x2), (y1, y2)))).
# X is a n x 2 matrix if (x, y) coordinates.
def build_theta(X, scale=1):
    # Get K(X, X).
    K = rbf_kernel(X, X, scale)

    # Isolate coordinates.
    xcoords = X[:, 0]
    ycoords = X[:, 1]
    dx = np.subtract.outer(xcoords, xcoords)
    dy = np.subtract.outer(ycoords, ycoords)

    # Get constants.
    inv_scale2 = 1 / (scale ** 2)
    inv_scale4 = inv_scale2 ** 2
    
    # Make the 4 n x n block matrices.
    top_left = (K * inv_scale2) * (1 - inv_scale2 * dx ** 2)
    top_right = -inv_scale4 * dx * dy * K
    bottom_left = top_right.copy().T
    bottom_right = (K * inv_scale2) * (1 - inv_scale2 * dy ** 2)

    # Stack into 2n x 2n matrix.
    Theta = np.block([[top_left, top_right], [bottom_left, bottom_right]])
    return Theta

# Find z to minimize ||Theta @ z - g||^2 + (lambda / 2) * z^T Theta^-1 z.
# Then alpha = Theta^-1 z.
def compute_alpha(Theta, g, _lambda):
    # Solve for z.
    Theta2 = Theta @ Theta
    Theta3 = Theta2 @ Theta
    Id = np.eye(Theta.shape[0])
    z = np.linalg.solve(2 * Theta3 + _lambda * Id, 2 * Theta2 @ g)

    # Solve for alpha.
    alpha = np.linalg.solve(Theta, z)
    return alpha

def eval_model(Xeval, X, alpha, scale):
    # X is our n x 2 (x, y) data points.
    # Xeval is our m x 2 (x, y) points we want to evaluate our model.
    K = rbf_kernel(Xeval, X, scale)

    # Get coords.
    xcoords_eval = Xeval[:, 0]
    xcoords = X[:, 0]
    ycoords_eval = Xeval[:, 1]
    ycoords = X[:, 1]

    # Get diffs.
    xdiff = np.subtract.outer(xcoords_eval, xcoords)
    ydiff = np.subtract.outer(ycoords_eval, ycoords)

    # Get G.
    G_top = -(xdiff / (scale ** 2)) * K
    G_bottom = -(ydiff / (scale ** 2)) * K
    G = np.vstack([G_top.T, G_bottom.T])

    # Get evaluation.
    _eval = alpha @ G

    return _eval

In [ ]:
n = 100
scale = 1
D = get_coordinates_uniform(radius=0.5, num_each_dir=n)
Theta = build_theta(X=D, scale=scale)
show(Theta)
[zf, g] = test_surface_data(D=D, noise=0)
alpha = compute_alpha(Theta=Theta, g=g, _lambda=0)

In [ ]:
surf = eval_model(Xeval=D, X=D, alpha=alpha, scale=scale)
surf = surf.reshape((n, n))
show(surf)

In [ ]:
surf_true = zf.reshape((n, n))
show(surf_true)